In [1]:
import numpy as np
import math
import time

In [ ]:
Number_of_particles = 1000
Accuracy = 5
Noise = 0. # degree


ParticlesPerUnit = 100
RadiusSystem = int(np.sqrt(Number_of_particles/ParticlesPerUnit))+1
RatioOfInteraction = 1.
dt = 1. / 30
#(np.power(Number_of_particles,0.33))
print('RadiusSystem = {}, RatioOfInteraction = {}, Density = {}, dt = {}'.format(RadiusSystem , RatioOfInteraction, ParticlesPerUnit, dt))

TimesRadius = 10
radius_step = RadiusSystem/TimesRadius

density = open('D:\\DENSITYV.txt', 'w')
module = open('D:\\MODULEV.txt', 'w')
momentum = open('D:\\MOMENTUMV.txt', 'w')

#result = open('D:\\RESULT.txt', 'w')
density.write('time\tstep\tdensv\n')
module.write('time\tstep\tmodulev\n')
momentum.write('time\tstep\tmomentumv\n')
#result.write('time\tX\tY\tVx\tVy\n')



class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[-10, 10, -10, 10],
                 noise=2 * np.pi,
                 RatioOfInteraction=1.,
                 RadiusSystem=1.):
        global Number_of_particles,Accuracy,TimesRadius,radius_step
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise * np.pi / 180
        self.noise_action = 0
        self.state = self.init_state.copy()
        self.state1 = self.init_state.copy()
        self.state_col = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt * np.ones(len(self.state[:, 0]))
        self.SQRT = np.ones(len(self.state[:, 0]))
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.mc = np.arange(3)
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 + 1)

        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b / self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)
        
    
    def step(self, dt):
        L0 = 1e5
        L100 = 0.
        Number_of_steps = 0
        i0=0
        while np.abs(L0 - L100) > 1/np.sqrt(Number_of_particles):
            start = time.clock()
            #for i in range(Number_of_particles):
            #   result.write('%.4f\t%.4f\t%.4f\t%.4f\t%.4f\n' % (self.time_elapsed , self.state[i,0] , self.state[i,1],self.state[i,2] , self.state[i,3]))
            self.time_elapsed += dt

            if Number_of_steps%Accuracy == Accuracy-1 :
                L100 = sum(np.abs((self.state[:, 0] * self.state[:, 3] - self.state[:, 1] * self.state[:, 2]) / np.hypot(self.state[:, 0], self.state[:, 1])))

            if  Number_of_steps%Accuracy == 0:
                L0 = sum(np.abs((self.state[:, 0] * self.state[:, 3] - self.state[:, 1] * self.state[:, 2]) / np.hypot(self.state[:, 0],self.state[:, 1])))

            print(L0,L100,Number_of_steps)
            Number_of_steps +=1
            print(self.time_elapsed)

            self.block[:] = -1
            self.state1 = self.state.copy()
            self.state_col = self.state.copy()
            self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
            self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
            self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))
            self.Amount_Particle = np.zeros(TimesRadius)
            self.MODULE_VELOCITY_HIST = np.zeros(TimesRadius)
            self.MOMENTUM_VELOCITY_HIST = np.zeros(TimesRadius)
            
            sx0 = 0
            sx1 = 0
            for i in range(len(self.state[:, 0])):
                self.lscl[i] = -1
                b = np.int(np.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * np.int(np.floor(np.abs(self.state[i, 1])))
                self.lscl[i] = self.block[b]
                self.block[b] = i

            for b0 in range(self.R2):
                f = int(self.block[b0])  
                while f != -1:
                    for b1 in range(8):
                        b2 = int(self.cellb[b0][b1])
                        j = int(self.block[b2])  
                        while j != -1:
                            dist = np.sqrt(
                                (self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)
                            self.noise_action = self.noise - 2 * self.noise * np.random.rand()

                            if dist < self.RatioOfInteraction and f != j:
                                sx0 += self.state_orts[j, 2]
                                sx1 += self.state_orts[j, 3]

                            j = int(self.lscl[j])
                    if (np.abs(sx0) < 0.00001) & (np.abs(sx1) < 0.00001):
                        self.state_col[:, 2:] = self.state_orts[:, 2:]
                        self.state_orts[f, 2] = self.state_col[f, 2] * np.cos(self.noise_action) - self.state_col[f, 3] * np.sin(self.noise_action)
                        self.state_orts[f, 3] = self.state_col[f, 2] * np.sin(self.noise_action) + self.state_col[f, 3] * np.cos(self.noise_action)
                    else:
                        self.state_orts[f, 2] = (sx0 * np.cos(self.noise_action) - sx1 * np.sin(self.noise_action)) / np.hypot(sx0, sx1)
                        self.state_orts[f, 3] = (sx0 * np.sin(self.noise_action) + sx1 * np.cos(self.noise_action)) / np.hypot(sx0, sx1)

                    sx0 = 0
                    sx1 = 0
                    
                    self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                    self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                    r = radius_step
                    
                    for i in range(TimesRadius):
                        
                        if (self.state[f, 0] ** 2 + self.state[f, 1] ** 2 < r ** 2) & (self.state[f, 0] ** 2 + self.state[f, 1] ** 2 > (r - radius_step) ** 2):
                            self.Amount_Particle[i] += 1 / Number_of_particles
                            self.MODULE_VELOCITY_HIST[i] += (self.state[f, 2] ** 2 + self.state[f, 3] ** 2) / Number_of_particles
                            self.MOMENTUM_VELOCITY_HIST[i] += np.abs((self.state[f, 0] * self.state[f, 3] - self.state[f, 1] * self.state[f, 2]) / (np.hypot(self.state[f, 0], self.state[f, 1]) * Number_of_particles))
                            
                        r += radius_step
                    

                    if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                        self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2]** 2 + self.state[f, 3] ** 2))
                        self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                        x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                        y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                        self.state_col[f, 2] = self.state[f, 2]
                        self.state_col[f, 3] = self.state[f, 3]

                        self.state[f, 3] = (self.state_col[f, 3] * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 *self.state_col[f, 2] * x01 * y01) / self.RadiusSystem ** 2
                        self.state[f, 2] = (self.state_col[f, 2] * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 *self.state_col[f, 3] * x01 * y01) / self.RadiusSystem ** 2

                        self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                        self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                        if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                            self.state[f, 0] = x01
                            self.state[f, 1] = y01
    
                    else:
                        self.state[f, 0] += dt * self.state[f, 2]
                        self.state[f, 1] += dt * self.state[f, 3]
                            
                    f = int(self.lscl[f])
            
            for q in range(TimesRadius):
                density.write('%.8f\t%.8f\t%.8f\n' % (self.time_elapsed , q , self.Amount_Particle[q]))
                module.write('%.8f\t%.8f\t%.8f\n'% (self.time_elapsed , q , self.MODULE_VELOCITY_HIST[q]))
                momentum.write('%.8f\t%.8f\t%.8f\n' % (self.time_elapsed , q , self.MOMENTUM_VELOCITY_HIST[q]))
            
            end = time.clock()
            print(str(round((end-start),3)) +' s')
            start =0
            end =0



#s = np.random.poisson(5, Number_of_particles)
#q = np.random.logseries(.4, Number_of_particles)
#P = 0.5*s+0.5*q
#np.random.poisson(5, Number_of_particles) + 10
#np.random.pareto(10, Number_of_particles)

distribution = np.random.poisson(1, Number_of_particles) + 4
max_element = np.amax(distribution)

P = 5.*distribution/max_element

degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()
Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State

circle = ParticleBox(init_state, noise=Noise, RatioOfInteraction=RatioOfInteraction,RadiusSystem = RadiusSystem)
circle.step(dt)
print('Done!')

RadiusSystem = 4, RatioOfInteraction = 1.0, Density = 100, dt = 0.03333333333333333
1734.46712535 0.0 0
0.03333333333333333
16.956 s
1734.46712535 0.0 1
0.06666666666666667
18.39 s
1734.46712535 0.0 2
0.1
14.607 s
1734.46712535 0.0 3
0.13333333333333333
13.853 s
1734.46712535 1478.83214087 4
0.16666666666666666
13.264 s
1493.09814023 1478.83214087 5
0.19999999999999998


# 100000 -> 16168.422 s

# Molecular field approach 

In [ ]:
Number_of_particles = 1000
Accuracy = 10
Noise = 0. # degree


ParticlesPerUnit = 100
RadiusSystem = int(np.sqrt(Number_of_particles/ParticlesPerUnit))+1
RatioOfInteraction = 1.
dt = 1. / 30
#(np.power(Number_of_particles,0.33))
print('RadiusSystem = {}, RatioOfInteraction = {}, Density = {}, dt = {}'.format(RadiusSystem , RatioOfInteraction, ParticlesPerUnit, dt))

TimesRadius = 10
radius_step = RadiusSystem/TimesRadius

density = open('D:\\DENSITYV_F.txt', 'w')
module = open('D:\\MODULEV_F.txt', 'w')
momentum = open('D:\\MOMENTUMV_F.txt', 'w')

#result = open('D:\\RESULT.txt', 'w')
density.write('time\tstep\tdensv\n')
module.write('time\tstep\tmodulev\n')
momentum.write('time\tstep\tmomentumv\n')
#result.write('time\tX\tY\tVx\tVy\n')



class ParticleBox:
    def __init__(self,
                 init_state=[[1, 0, 0, 1],
                             [-0.5, 0.5, 0.5, 0.5],
                             [-0.5, -0.5, -0.5, 0.5]],
                 bounds=[-10, 10, -10, 10],
                 noise=2 * np.pi,
                 RatioOfInteraction=1.,
                 RadiusSystem=1.):
        global Number_of_particles,Accuracy,TimesRadius,radius_step
        self.init_state = np.asarray(init_state, dtype=float)
        self.noise = noise * np.pi / 180
        self.noise_action = 0
        self.state = self.init_state.copy()
        self.state1 = self.init_state.copy()
        self.state_col = self.init_state.copy()
        self.state_orts = self.init_state.copy()
        self.mod = np.ones(len(self.state[:, 0]))
        self.time_elapsed = 0
        self.tau = dt * np.ones(len(self.state[:, 0]))
        self.SQRT = np.ones(len(self.state[:, 0]))
        self.RatioOfInteraction = RatioOfInteraction
        self.bounds = bounds
        self.RadiusSystem = RadiusSystem
        self.R2 = self.RadiusSystem ** 2
        self.lscl = np.empty(len(self.state[:, 0]))
        self.mc = np.arange(3)
        self.cellv0 = np.array(
            [[self.RadiusSystem - 1, self.RadiusSystem - 1], [0, self.RadiusSystem - 1], [1, self.RadiusSystem - 1],
             [self.RadiusSystem - 1, 0], [0, 0], [1, 0], [self.RadiusSystem - 1, 1], [0, 1], [1, 1]])
        self.cellb = np.empty([self.R2, 9])
        self.block = np.empty(self.R2 + 1)

        self.block[:] = -1
        self.lscl[:] = -1
        bx = 0
        by = 0
        for b in range(self.R2):
            for j in range(8):
                bx = self.cellv0[j, 0] + int(b % self.RadiusSystem)
                by = self.cellv0[j, 0] + int(b / self.RadiusSystem)
                self.cellb[b, j] = int(bx % self.RadiusSystem + (by % self.RadiusSystem) * self.RadiusSystem)
        
    
    def step(self, dt):
        L0 = 1e5
        L100 = 0.
        Number_of_steps = 0
        i0=0
        while np.abs(L0 - L100) > 1/np.sqrt(Number_of_particles):
            start = time.clock()
            #for i in range(Number_of_particles):
            #   result.write('%.4f\t%.4f\t%.4f\t%.4f\t%.4f\n' % (self.time_elapsed , self.state[i,0] , self.state[i,1],self.state[i,2] , self.state[i,3]))
            self.time_elapsed += dt

            if Number_of_steps%Accuracy == Accuracy-1 :
                L100 = sum(np.abs((self.state[:, 0] * self.state[:, 3] - self.state[:, 1] * self.state[:, 2]) / np.hypot(self.state[:, 0], self.state[:, 1])))

            if  Number_of_steps%Accuracy == 0:
                L0 = sum(np.abs((self.state[:, 0] * self.state[:, 3] - self.state[:, 1] * self.state[:, 2]) / np.hypot(self.state[:, 0],self.state[:, 1])))

            print(L0,L100,Number_of_steps)
            Number_of_steps +=1
            print(self.time_elapsed)

            self.block[:] = -1
            self.state1 = self.state.copy()
            self.state_col = self.state.copy()
            self.mod = np.hypot(self.state[:, 2], self.state[:, 3])
            self.state_orts[:, 2] = np.divide(self.state[:, 2], np.hypot(self.state[:, 2], self.state[:, 3]))
            self.state_orts[:, 3] = np.divide(self.state[:, 3], np.hypot(self.state[:, 2], self.state[:, 3]))
            self.Amount_Particle = np.zeros(TimesRadius)
            self.MODULE_VELOCITY_HIST = np.zeros(TimesRadius)
            self.MOMENTUM_VELOCITY_HIST = np.zeros(TimesRadius)
            
            sx0 = 0
            sx1 = 0
            for i in range(len(self.state[:, 0])):
                self.lscl[i] = -1
                b = np.int(np.floor(np.abs(self.state[i, 0]))) + self.RadiusSystem * np.int(np.floor(np.abs(self.state[i, 1])))
                self.lscl[i] = self.block[b]
                self.block[b] = i

            for b0 in range(self.R2):
                f = int(self.block[b0])  
                while f != -1:
                    for b1 in range(8):
                        b2 = int(self.cellb[b0][b1])
                        j = int(self.block[b2])  
                        while j != -1:
                            #dist = np.sqrt((self.state[f, 0] - self.state[j, 0]) ** 2 + (self.state[f, 1] - self.state[j, 1]) ** 2)
                            self.noise_action = self.noise - 2 * self.noise * np.random.rand()

                            #if dist < self.RatioOfInteraction and f != j:
                            sx0 += self.state_orts[j, 2]
                            sx1 += self.state_orts[j, 3]

                            j = int(self.lscl[j])
                    if (np.abs(sx0) < 0.00001) & (np.abs(sx1) < 0.00001):
                        self.state_col[:, 2:] = self.state_orts[:, 2:]
                        self.state_orts[f, 2] = self.state_col[f, 2] * np.cos(self.noise_action) - self.state_col[f, 3] * np.sin(self.noise_action)
                        self.state_orts[f, 3] = self.state_col[f, 2] * np.sin(self.noise_action) + self.state_col[f, 3] * np.cos(self.noise_action)
                    else:
                        self.state_orts[f, 2] = (sx0 * np.cos(self.noise_action) - sx1 * np.sin(self.noise_action)) / np.hypot(sx0, sx1)
                        self.state_orts[f, 3] = (sx0 * np.sin(self.noise_action) + sx1 * np.cos(self.noise_action)) / np.hypot(sx0, sx1)

                    sx0 = 0
                    sx1 = 0
                    
                    self.state[f, 2] = self.mod[f] * self.state_orts[f, 2]
                    self.state[f, 3] = self.mod[f] * self.state_orts[f, 3]

                    r = radius_step
                    
                    for i in range(TimesRadius):
                        
                        if (self.state[f, 0] ** 2 + self.state[f, 1] ** 2 < r ** 2) & (self.state[f, 0] ** 2 + self.state[f, 1] ** 2 > (r - radius_step) ** 2):
                            self.Amount_Particle[i] += 1 / Number_of_particles
                            self.MODULE_VELOCITY_HIST[i] += (self.state[f, 2] ** 2 + self.state[f, 3] ** 2) / Number_of_particles
                            self.MOMENTUM_VELOCITY_HIST[i] += np.abs((self.state[f, 0] * self.state[f, 3] - self.state[f, 1] * self.state[f, 2]) / (np.hypot(self.state[f, 0], self.state[f, 1]) * Number_of_particles))
                            
                        r += radius_step
                    

                    if (self.state[f, 0] + self.state[f, 2] * dt) ** 2 + (self.state[f, 1] + self.state[f, 3] * dt) ** 2 >= self.RadiusSystem ** 2:

                        self.SQRT = ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)) ** 2 + ((self.RadiusSystem ** 2 - self.state[f, 1] ** 2 - self.state[f, 0] ** 2) / (self.state[f, 2]** 2 + self.state[f, 3] ** 2))
                        self.tau = np.abs(np.sqrt(np.abs(self.SQRT)) - ((self.state[f, 0] * self.state[f, 2] + self.state[f, 1] * self.state[f, 3]) / (self.state[f, 2] ** 2 + self.state[f, 3] ** 2)))

                        x01 = self.state[f, 0] + self.state[f, 2] * self.tau
                        y01 = self.state[f, 1] + self.state[f, 3] * self.tau

                        self.state_col[f, 2] = self.state[f, 2]
                        self.state_col[f, 3] = self.state[f, 3]

                        self.state[f, 3] = (self.state_col[f, 3] * (self.RadiusSystem ** 2 - 2. * y01 ** 2) - 2 *self.state_col[f, 2] * x01 * y01) / self.RadiusSystem ** 2
                        self.state[f, 2] = (self.state_col[f, 2] * (self.RadiusSystem ** 2 - 2. * x01 ** 2) - 2 *self.state_col[f, 3] * x01 * y01) / self.RadiusSystem ** 2

                        self.state[f, 0] = self.state[f, 0] + self.state[f, 2] * (dt - self.tau)
                        self.state[f, 1] = self.state[f, 1] + self.state[f, 3] * (dt - self.tau)

                        if self.state[f, 0] ** 2 + self.state[f, 1] ** 2 >= self.RadiusSystem ** 2:
                            self.state[f, 0] = x01
                            self.state[f, 1] = y01
    
                    else:
                        self.state[f, 0] += dt * self.state[f, 2]
                        self.state[f, 1] += dt * self.state[f, 3]
                            
                    f = int(self.lscl[f])
            
            for q in range(TimesRadius):
                density.write('%.8f\t%.8f\t%.8f\n' % (self.time_elapsed , q , self.Amount_Particle[q]))
                module.write('%.8f\t%.8f\t%.8f\n'% (self.time_elapsed , q , self.MODULE_VELOCITY_HIST[q]))
                momentum.write('%.8f\t%.8f\t%.8f\n' % (self.time_elapsed , q , self.MOMENTUM_VELOCITY_HIST[q]))
            
            end = time.clock()
            print(str(round((end-start),3)) +' s')
            start =0
            end =0



#s = np.random.poisson(5, Number_of_particles)
#q = np.random.logseries(.4, Number_of_particles)
#P = 0.5*s+0.5*q
#np.random.poisson(5, Number_of_particles) + 10
#np.random.pareto(10, Number_of_particles)

distribution = np.random.poisson(1, Number_of_particles) + 4
max_element = np.amax(distribution)

P = 5.*distribution/max_element

degree = 2. * np.pi * np.random.random((Number_of_particles, 1))
degree_pos = 2. * np.pi * np.random.random((Number_of_particles, 1))
R0 = np.random.random((Number_of_particles, 1))
Start_velocity_x = (P * np.cos(degree).transpose()).transpose()
Start_velocity_y = (P * np.sin(degree).transpose()).transpose()
Start_coord_x = RadiusSystem * R0 * np.cos(degree_pos)
Start_coord_y = RadiusSystem * R0 * np.sin(degree_pos)

Velocity = np.column_stack((Start_velocity_x, Start_velocity_y))
Coord = np.column_stack((Start_coord_x, Start_coord_y))
Random_State = np.random.random((Number_of_particles, 4))
Random_State[:, 2:] = Velocity
Random_State[:, :2] = Coord

init_state = Random_State

 
circle = ParticleBox(init_state, noise=Noise, RatioOfInteraction=RatioOfInteraction,RadiusSystem = RadiusSystem)
circle.step(dt)
print('Done!')